# 🎲 The Black-Jack — *Classic Before Quantum*  

Hello everyone! 👋  

Before diving into **Quantum Blackjack**, it’s good practice to build the game in its **classical form** first.  
In this notebook, we’ll walk through how to implement Blackjack step by step — the traditional way 🃏


<div style="margin:1.2em 0; background:#f8fafc; color:#111827; border:1px solid #cbd5e1; border-radius:10px; padding:16px 20px; font-size:15.5px; line-height:1.55; font-family:Segoe UI, Roboto, sans-serif;">
  <div style="font-weight:700; color:#1e293b; margin-bottom:.5rem; font-size:16.5px;">
    📌 Step 1 — Implementing the Cards
  </div>
  <p style="margin:0 0 .6rem 0;">
    In this step, you’ll define the <b>Card</b> object, which is the foundation of our Blackjack game. Follow these tasks:
  </p>
  <ol style="margin:.2rem 0 0 1.4rem; padding:0;">
    <li>Define a <b>Card</b> class.</li>
    <li>Add <code>rank</code> and <code>suit</code> as attributes (set them in the constructor).</li>
    <li>Inside the class, implement a <code>value()</code> method that returns the numeric value of the card (1–11).</li>
    <li>Remember that <code>value()</code> is trivial when rank is just a number but you need to handle cases such as J,Q,K.</li>
    
  </ol>
</div>


In [ ]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def value(self):
        try:
            if self.rank in ['J', 'Q', 'K']:
                return 10
            if self.rank == 'A':
                return 11  # or 1 based on game rules
            return int(self.rank)  # Assuming rank is a number string for 2-10
        except ValueError:
            raise ValueError("Invalid rank value")



#PROGRAM TEST
card = Card(4, 'Hearts')
print(card.value())  # Should print 4

<details style="margin:1.2em 0; border:1px solid #f8fafc; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#f1f5f9; color:#1e293b; font-weight:600; font-size:15px;">
    💡 Not sure about how to implmenet the cards? <i>Click here for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#ffffff; color:#111827; font-size:14.5px; line-height:1.55;">
    Each card has two key parts:
    <ul style="margin:.4rem 0 0 1.2rem;">
      <li><b>Rank</b> → the number or face (2–10, J, Q, K, A)</li>
      <li><b>Suit</b> → hearts ♥, diamonds ♦, clubs ♣, or spades ♠</li>
      <li><code>['J', 'Q', 'K']</code> has value 10</li>
      <li><code>A</code> is a special card which can be evaluated as 1-11, for now define it as 11 but keep in mind this stiuation when evaluating the hand!</li>
    </ul>
    In code, you’ll represent these as attributes of the <code>Card</code> class.
  </div>
</details>


<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    💡 Need help for <code>value()</code> method? <i>Click to see pseudocode</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    <pre style="margin:0; font-size:14px; line-height:1.5; background:#f1f5f9; border:1px solid #e5e7eb; border-radius:6px; padding:10px; overflow:auto; color:#1f2937;">
<code>FUNCTION value(card):
    IF rank IN {J, Q, K} THEN
        RETURN 10
    ELSE IF rank = A THEN
        RETURN 11   // will be adjusted later if needed
    ELSE
        RETURN integer(rank)</code></pre>
  </div>
</details>


<div style="margin:1.2em 0; background:#f8fafc; color:#111827; border:1px solid #cbd5e1; border-radius:10px; padding:16px 20px; font-size:15.5px; line-height:1.55; font-family:Segoe UI, Roboto, sans-serif;">
  <div style="font-weight:700; color:#1e293b; margin-bottom:.5rem; font-size:16.5px;">
    📌 Step 2 — Implementing the Deck
  </div>
  <p style="margin:0 0 .6rem 0;">
    Now that we have a single <b>Card</b>, the next step is to build a <b>Deck</b> of 52 cards (or multiple decks if desired). Follow these tasks:
  </p>
  <ol style="margin:.2rem 0 0 1.4rem; padding:0;">
    <li>Define a <b>Deck</b> class with an optional parameter <code>n_decks</code> (default = 1).</li>
    <li>In the constructor, generate all cards:
      <ul style="margin:.2rem 0 0 1.2rem;">
        <li>Iterate over the four suits: ♥ Hearts, ♦ Diamonds, ♣ Clubs, ♠ Spades.</li>
        <li>For each suit, add number cards (2–10) and face cards (J, Q, K, A).</li>
      </ul>
    </li>
    <li>Store all cards in a list <code>self.cards</code>.</li>
    <li>Add a <code>shuffle()</code> method to randomize the deck.</li>
    <li>Add a <code>draw()</code> method to remove and return the top card (or <code>None</code> if empty).</li>
  </ol>
</div>


In [ ]:
import random

class Deck:
    def __init__(self, n_decks=1):
        self.cards = []
        for _ in range(n_decks):
            for suit in ['Hearts', 'Diamonds', 'Clubs', 'Spades']:
                for rank in range(2, 11):
                    self.cards.append(Card(rank, suit))
                for rank in ['J', 'Q', 'K', 'A']:
                    self.cards.append(Card(rank, suit))
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def draw(self):
        return self.cards.pop() if self.cards else None
    

#PROGRAM TEST
deck = Deck()
print(len(deck.cards))  # Should print 52 for a single deck
card = deck.draw()
print(card.rank, card.suit)  # Should print the rank and suit of the drawn card

<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    🏗️ Need help with the constructor? <i>Click here for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    The <code>__init__</code> method builds a full deck of cards:
    <ul style="margin:.4rem 0 0 1.2rem;">
      <li>Start with an empty list <code>self.cards</code>.</li>
      <li>Repeat the process for <code>n_decks</code> times.</li>
      <li>Loop through the four suits: ♥ Hearts, ♦ Diamonds, ♣ Clubs, ♠ Spades.</li>
      <li>For each suit:
        <ul style="margin:.3rem 0 0 1.2rem;">
          <li>Add cards with ranks <code>2–10</code>.</li>
          <li>Add the face cards <code>J, Q, K</code> and the <code>A</code>.</li>
        </ul>
      </li>
    </ul>
    <pre style="margin:.8rem 0 0 0; font-size:14px; line-height:1.5; background:#f1f5f9; border:1px solid #e5e7eb; border-radius:6px; padding:10px; overflow:auto; color:#1f2937;">
<code>self.cards = []
for suit in ['Hearts','Diamonds','Clubs','Spades']:
    for rank in range(2, 11):
        self.cards.append(Card(rank, suit))
    for rank in ['J','Q','K','A']:
        self.cards.append(Card(rank, suit))</code></pre>
  </div>
</details>


<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    🔄 Not sure how to shuffle the deck? <i>Click here for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    You can shuffle a list of cards using Python’s built-in:
    <pre style="margin:0; font-size:14px; line-height:1.5; background:#f1f5f9; border:1px solid #e5e7eb; border-radius:6px; padding:10px; overflow:auto; color:#1f2937;">
<code>import random
random.shuffle(deck.cards)</code></pre>
  </div>
</details>


<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    🃏 How to draw a card? <i>Click here for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    In Python, lists behave like stacks. Use <code>.pop()</code> to remove and return the last element:
    <pre style="margin:0; font-size:14px; line-height:1.5; background:#f1f5f9; border:1px solid #e5e7eb; border-radius:6px; padding:10px; overflow:auto; color:#1f2937;">
<code>card = deck.cards.pop()
print(card.rank, card.suit)</code></pre>
  </div>
</details>


<div style="margin:1.2em 0; background:#f8fafc; color:#111827; border:1px solid #cbd5e1; border-radius:10px; padding:16px 20px; font-size:15.5px; line-height:1.55; font-family:Segoe UI, Roboto, sans-serif;">
  <div style="font-weight:700; color:#1e293b; margin-bottom:.5rem; font-size:16.5px;">
    📌 Step 3 — Implementing the Hand
  </div>
  <p style="margin:0 0 .6rem 0;">
    Now we need a <b>Hand</b> class to represent the cards a player (or dealer) is holding. It should support adding cards and evaluating the hand’s value.
  </p>
  <ol style="margin:.2rem 0 0 1.4rem; padding:0;">
    <li>Initialize with an empty list <code>self.cards</code>.</li>
    <li>Implement <code>add_card(card)</code> to add a <b>Card</b> to the hand.</li>
    <li>Create <code>best_value()</code>:
      <ul style="margin:.2rem 0 0 1.2rem;">
        <li>Sum the values of all cards.</li>
        <li>Figure out how to handle the aces properly.</li>
      </ul>
    </li>
    <li>Add <code>is_blackjack()</code>: returns <code>True</code> if the hand has exactly 2 cards totaling 21.</li>
    <li>Add <code>is_bust()</code>: returns <code>True</code> if the total goes over 21.</li>
  </ol>
</div>


In [ ]:
class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def best_value(self):
        # Calculate the best total value of the hand, considering Aces as 1 or 11
        total = sum(card.value() for card in self.cards)
        aces = sum(1 for card in self.cards if card.rank == 'A')
        while total > 21 and aces:
            total -= 10
            aces -= 1
        return total

    def is_blackjack(self):
        return len(self.cards) == 2 and self.best_value() == 21

    def is_bust(self):
        return self.best_value() > 21


#PROGRAM TEST
hand = Hand()
hand.add_card(Card('A', 'Hearts'))
hand.add_card(Card(10, 'Diamonds'))
hand.add_card(Card(10, 'Clubs'))
print(hand.best_value())  # Should print 21

<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    ♠️ How to handle Aces (1 or 11)? <i>Click for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    Start by treating every Ace as 11.  
    If the total value is above 21, reduce some Aces from 11 → 1 (subtract 10 each time) until the hand is safe or no Aces remain.
  
  </div>
</details>


<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    🃏 How to check for Blackjack? <i>Click for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    You already implemented <code>best_value()</code>, just use it to check whether your hand is = 21 and check how many cards in your hand. If best value is 21 and you have 2 cards, that is a blackjack!
  </div>
</details>


<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    💥 How to detect a Bust? <i>Click for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    You already implemented <code>best_value()</code>, just use it to check whether your hand is > 21 or not.
  </div>
</details>


<div style="margin:1.2em 0; background:#f8fafc; color:#111827; border:1px solid #cbd5e1; border-radius:10px; padding:16px 20px; font-size:15.5px; line-height:1.55; font-family:Segoe UI, Roboto, sans-serif;">
  <div style="font-weight:700; color:#1e293b; margin-bottom:.5rem; font-size:16.5px;">
    📌 Step 4 — Implementing the Player
  </div>
  <p style="margin:0 0 .6rem 0;">
    Now we define a <b>Player</b> class to represent each participant in the game. A player needs a hand, the ability to draw cards, and a way to make decisions (hit or stand).
  </p>
  <ol style="margin:.2rem 0 0 1.4rem; padding:0;">
    <li>Create a <b>Player</b> class with:
      <ul style="margin:.2rem 0 0 1.2rem;">
        <li>A <code>name</code> attribute to identify the player.</li>
        <li>A <code>hand</code> attribute (use the <code>Hand()</code> class we previously defined) initialized as an empty <b>Hand</b>.</li>
      </ul>
    </li>
    <li>Implement a <code>draw(deck)</code> method:
      <ul style="margin:.2rem 0 0 1.2rem;">
        <li>Draw the top card from the deck. (Remember that is just a method of <code>hand</code>)</li>
        <li>Do not forget to handle the probability of deck might be empty.</li>
      </ul>
    </li>
    <li>Implement a <code>decide()</code> method:
      <ul style="margin:.2rem 0 0 1.2rem;">
        <li>Prompt the player to choose between <b>hit</b> or <b>stand</b>.</li>
        <li>Validate the input — repeat until the player enters a valid choice.</li>
        <li>Return the chosen action in string format.<code>"hit"</code> or <code>"stand"</code></li>
      </ul>
    </li>
  </ol>
</div>

In [ ]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()

    def draw(self, deck):
        card = deck.draw()
        if card:
            self.hand.add_card(card)
        return card

    def decide(self):
        # Ask for input (hit/stand)
        action = input(f"{self.name}, do you want to hit or stand? ").strip().lower()
        while action not in ['hit', 'stand']:
            print("Invalid input. Please enter 'hit' or 'stand'.")
            action = input(f"{self.name}, do you want to hit or stand? ").strip().lower()
        return action


#PROGRAM TEST
player = Player("Alice")
deck = Deck()
player.draw(deck)
print(player.hand.cards[0].rank, player.hand.cards[0].suit)  # Should print the rank and suit of the drawn card
player.decide()  # Should prompt for input

<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    🃏 How does <code>draw()</code> work? <i>Click for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    The <code>Player</code> doesn’t create cards itself — it asks the <code>Deck</code> for one, so check how did you implemented your <code>Deck.draw()</code>and then:
    <pre style="margin:.8rem 0 0; font-size:14px; background:#f1f5f9; border:1px solid #e5e7eb; border-radius:6px; padding:10px; overflow:auto; color:#1f2937;">
<code>card = deck.draw()
if card:
    self.hand.add_card(card)</code></pre>
    This way, the card is removed from the deck and added to the player’s hand.
  </div>
</details>


<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    ✅ How to validate the choice? <i>Click for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    Make sure only <code>"hit"</code> or <code>"stand"</code> are accepted:
    <pre style="margin:.8rem 0 0; font-size:14px; background:#f1f5f9; border:1px solid #e5e7eb; border-radius:6px; padding:10px; overflow:auto; color:#1f2937;">
<code>while action not in ['hit', 'stand']:
    print("Invalid input. Please enter 'hit' or 'stand'.")
    action = input("Hit or stand? ").strip().lower()</code></pre>
    This loop keeps asking until the player enters a valid choice.
  </div>
</details>


<div style="margin:1.2em 0; background:#f8fafc; color:#111827; border:1px solid #cbd5e1; border-radius:10px; padding:16px 20px; font-size:15.5px; line-height:1.55; font-family:Segoe UI, Roboto, sans-serif;">
  <div style="font-weight:700; color:#1e293b; margin-bottom:.5rem; font-size:16.5px;">
    📌 Step 5 — Implementing the Dealer
  </div>
  <p style="margin:0 0 .6rem 0;">
    The <b>Dealer</b> is a special type of <code>Player</code>. Instead of asking for user input, the dealer follows fixed rules: keep drawing cards until the total is 17 or higher.
  </p>
  <ol style="margin:.2rem 0 0 1.4rem; padding:0;">
    <li>Create a <b>Dealer</b> class that <b>inherits</b> from <code>Player</code>.</li>
    <li>Implement a <code>play(deck)</code> method:
      <ul style="margin:.2rem 0 0 1.2rem;">
        <li>While the dealer’s hand total is less than 17 → keep drawing cards.</li>
        <li>After each draw, you can print a message (e.g., <code>"Dealer drew a card."</code>).</li>
      </ul>
    </li>
    <li>Once the dealer reaches 17 or more, stop drawing.</li>
  </ol>
</div>


In [ ]:
class Dealer(Player):
    def play(self, deck):
        # Dealer keeps hitting until 17 or higher
        while self.hand.best_value() < 17:
            self.draw(deck)
            print(f"{self.name} drew a card.")


#PROGRAM TEST
dealer = Dealer("Dealer")
dealer.play(deck)
for card in dealer.hand.cards:
    print(card.rank, card.suit)  # Should print all cards in dealer's hand

<details style="margin:1.2em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    🃏 Why does <code>Dealer</code> inherit from <code>Player</code>? <i>Click for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    The <b>Dealer</b> is also a kind of player:  
    <ul style="margin:.4rem 0 0 1.2rem;">
      <li>They have a <code>name</code> and a <code>Hand</code>, just like a Player.</li>
      <li>They can <code>draw()</code> cards from the Deck.</li>
      <li>But their decisions are automatic — they don’t use <code>decide()</code>.</li>
    </ul>
    By inheriting from <code>Player</code>, the <code>Dealer</code> reuses existing code and only needs to define its special behavior.
  </div>
</details>


<div style="margin:1.2em 0; background:#f8fafc; color:#111827; border:1px solid #cbd5e1; border-radius:10px; padding:16px 20px; font-size:15.5px; line-height:1.55; font-family:Segoe UI, Roboto, sans-serif;">
  <div style="font-weight:700; color:#1e293b; margin-bottom:.5rem; font-size:16.5px;">
    📌 Step 6 — Implementing the Game
  </div>
  <p style="margin:0 0 .6rem 0;">
    The <b>Game</b> class coordinates a single round of Blackjack: setup, initial deal, the player’s turn, the dealer’s turn, and deciding the winner.
  </p>
  <ol style="margin:.2rem 0 0 1.4rem; padding:0;">
    <li>Initialize: create and shuffle a <code>Deck</code>, and create <code>Player</code> and <code>Dealer</code>.</li>
    <li><code>deal_initial()</code>: deal two cards each (player first); show player total and dealer’s visible card; check player blackjack.</li>
    <li><code>player_turn()</code>: prompt for <b>hit</b>/<b>stand</b>; on hit → draw &amp; re-check for bust; stop when stand or bust.</li>
    <li><code>dealer_turn()</code>: dealer auto-plays (hits to 17+).</li>
    <li><code>determine_winner()</code>: compare totals with bust rules; print outcome.</li>
    <li><code>start()</code>: run the sequence; if no player blackjack, play the round and determine the winner.</li>
  </ol>
</div>


In [ ]:
class Game:
    def __init__(self):
        self.deck = Deck()
        self.deck.shuffle()
        self.player = Player("Player")
        self.dealer = Dealer("Dealer")

    def deal_initial(self):
        for _ in range(2):
            card = self.player.draw(self.deck)
            print(f"{self.player.name} drew {card.rank} of {card.suit}. Current value: {self.player.hand.best_value()}")
            self.dealer.draw(self.deck)
        print(f"{self.player.name}'s initial hand value: {self.player.hand.best_value()}")
        print(f"{self.dealer.name}'s visible card: {self.dealer.hand.cards[0].rank} of {self.dealer.hand.cards[0].suit}")
        if self.player.hand.is_blackjack():
            print(f"{self.player.name} has a blackjack! {self.player.name} wins!")

    def player_turn(self):
        action = self.player.decide()
        while action == 'hit':
            card = self.player.draw(self.deck)
            print(f"{self.player.name} drew {card.rank} of {card.suit}. Current value: {self.player.hand.best_value()}")
            if self.player.hand.is_bust():
                print(f"{self.player.name} busts with value {self.player.hand.best_value()}!")
                return
            elif self.player.hand.best_value() == 21:
                return
            action = self.player.decide()
        # Player stands
        print(f"{self.player.name} stands with value {self.player.hand.best_value()}.")

    def dealer_turn(self):
        self.dealer.play(self.deck)
        print(f"{self.dealer.name} stands with value {self.dealer.hand.best_value()}.")

    def determine_winner(self):
        player_value = self.player.hand.best_value()
        dealer_value = self.dealer.hand.best_value()

        if self.player.hand.is_bust():
            print("Dealer wins! Player busted.")
        elif self.dealer.hand.is_bust():
            print("Player wins! Dealer busted.")
        elif player_value > dealer_value:
            print("Player wins!")
        elif dealer_value > player_value:
            print("Dealer wins!")
        else:
            print("It's a tie!")

    def play_round(self):
        self.player_turn()
        if not self.player.hand.is_bust():
            self.dealer_turn()

    def start(self):
        self.deal_initial()
        if not self.player.hand.is_blackjack():
            self.play_round()
            self.determine_winner()

game = Game()
game.start()

<details style="margin:1.0em 0; border:1px solid #cbd5e1; border-radius:10px; overflow:hidden; font-family:Segoe UI, Roboto, sans-serif;">
  <summary style="cursor:pointer; padding:10px 14px; background:#e2e8f0; color:#1e293b; font-weight:600; font-size:15px;">
    🃏 How should the initial deal work? <i>Click for a hint</i>
  </summary>
  <div style="padding:12px 16px; background:#f8fafc; color:#111827; font-size:14.5px; line-height:1.55;">
    Deal <b>two rounds</b> of cards: player draws, then dealer draws. After the loop:
    <ul style="margin:.4rem 0 0 1.2rem;">
      <li>Print the player’s total.</li>
      <li>Reveal the dealer’s <b>first</b> (visible) card.</li>
      <li>If the player has a blackjack, announce and stop later via <code>start()</code>.</li>
    </ul>
    <pre style="margin:.8rem 0 0; font-size:14px; background:#f1f5f9; border:1px solid #e5e7eb; border-radius:6px; padding:10px; overflow:auto; color: #1f2937;">
<code>for _ in range(2):
    card = self.player.draw(self.deck)
    print(f"{self.player.name} drew {card.rank} of {card.suit}. Current value: {self.player.hand.best_value()}")
    self.dealer.draw(self.deck)

print(f"{self.player.name}'s initial hand value: {self.player.hand.best_value()}")
print(f"{self.dealer.name}'s visible card: {self.dealer.hand.cards[0].rank} of {self.dealer.hand.cards[0].suit}")

if self.player.hand.is_blackjack():
    print(f"{self.player.name} has a blackjack! {self.player.name} wins!")</code></pre>
  </div>
</details>


<div style="margin:1.2em 0; background:#f8fafc; color:#111827; border:1px solid #cbd5e1; border-radius:10px; padding:18px 22px; font-size:15.5px; line-height:1.55; font-family:Segoe UI, Roboto, sans-serif;">
  <div style="font-weight:700; color:#1e293b; margin-bottom:.6rem; font-size:17px;">
    📦 Putting It All Together — The Game Loop
  </div>
  <p style="margin:0 0 .8rem 0;">
    You’ve now implemented all the building blocks of Blackjack: <b>Card</b>, <b>Deck</b>, <b>Hand</b>, <b>Player</b>, <b>Dealer</b>, and <b>Game</b>.  
    Below is the final <code>main()</code> function that ties everything together and lets you play multiple rounds:
  </p>

</div>


```python
def main():
    game = Game()  # create a new game
    
    while True:
        print("\n--- New Round ---")
        game.start()  # one full round of Blackjack
        
        # Ask player if they want to play again
        again = input("Play again? (y/n): ").lower()
        if again != "y":
            print("Thanks for playing! 👋")
            break


# Start the game loop
if __name__ == "__main__":
    main()


In [ ]:
def main():
    game = Game()  # create a new game
    
    while True:
        print("\n--- New Round ---")
        game.start()  # one full round of Blackjack
        
        # Ask player if they want to play again
        again = input("Play again? (y/n): ").lower()
        if again != "y":
            print("Thanks for playing! 👋")
            break


# Start the game loop
if __name__ == "__main__":
    main()
